In [ ]:
import os
import json
import pytz
import requests
import tempfile
import numpy as np
from datetime import datetime, timedelta, timezone
import pandas as pd
from html.parser import HTMLParser
import xml.etree.ElementTree as ET

In [ ]:
bucket = "https://alplakes-eawag.s3.eu-central-1.amazonaws.com"
bucket_key = bucket.split(".")[0].split("//")[1]

In [ ]:
class boto3(object):
    def __init__(self):
        self.intialise = ""
    def upload_file(self, file, bucket, path):
        print(file, bucket, path)

s3 = boto3()

In [ ]:
def ch1903_plus_to_latlng(x, y):
    x_aux = (x - 2600000) / 1000000
    y_aux = (y - 1200000) / 1000000
    lat = 16.9023892 + 3.238272 * y_aux - 0.270978 * x_aux ** 2 - 0.002528 * y_aux ** 2 - 0.0447 * x_aux ** 2 * y_aux - 0.014 * y_aux ** 3
    lng = 2.6779094 + 4.728982 * x_aux + 0.791484 * x_aux * y_aux + 0.1306 * x_aux * y_aux ** 2 - 0.0436 * x_aux ** 3
    lat = (lat * 100) / 36
    lng = (lng * 100) / 36
    return lat, lng

In [ ]:
class TableHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.rows = []
        self.current_row = []
        self.in_td = False

    def handle_starttag(self, tag, attrs):
        if tag == 'td':
            self.in_td = True

    def handle_endtag(self, tag):
        if tag == 'tr':
            if self.current_row:
                self.rows.append(self.current_row)
                self.current_row = []
        elif tag == 'td':
            self.in_td = False

    def handle_data(self, data):
        if self.in_td:
            self.current_row.append(data.strip())

def parse_html_table(html_table):
    parser = TableHTMLParser()
    parser.feed(html_table)
    df = pd.DataFrame(parser.rows)
    return df

In [ ]:
class CustomHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.root = ET.Element("root")
        self.current = self.root
        self.stack = [self.root]  # Stack to keep track of parent elements

    def handle_starttag(self, tag, attrs):
        attrs_dict = dict(attrs)
        element = ET.SubElement(self.current, tag, attrs_dict)
        self.stack.append(self.current)  # Push current element to stack
        self.current = element

    def handle_endtag(self, tag):
        self.current = self.stack.pop()  # Pop from stack to move to the parent

    def handle_data(self, data):
        if self.current.text is None:
            self.current.text = data
        else:
            self.current.text += data

def parse_html(html_string):
    parser = CustomHTMLParser()
    parser.feed(html_string)
    return parser.root

def html_find_all(element, tag=None, class_name=None, attributes=None):
    def match_attributes(el, attributes):
        if not attributes:
            return True
        for attr, value in attributes.items():
            if el.get(attr) != value:
                return False
        return True

    def match_class_name(el, class_name):
        if class_name is None:
            return True
        return class_name in el.get('class', '').split()

    # Recursively search for matching elements
    def find_all_recursive(el, matches):
        if ((tag is None or el.tag == tag) and
            match_class_name(el, class_name) and
            match_attributes(el, attributes)):
            matches.append(el)
        for child in el:
            find_all_recursive(child, matches)

    matches = []
    find_all_recursive(element, matches)
    return matches

In [ ]:
features = []
failed = []
swiss_timezone = pytz.timezone("Europe/Zurich")
min_date = (datetime.now() - timedelta(days=14)).timestamp()

BAFU

In [ ]:
try:
    lookup = {"2606": "geneva", "2104": "walensee", "2152": "lucerne", "2030": "thun", "2457": "brienz"}
    response = requests.get("https://www.hydrodaten.admin.ch/web-hydro-maps/hydro_sensor_temperature.geojson")
    if response.status_code == 200:
        for f in response.json()["features"]:
            lat, lng = ch1903_plus_to_latlng(f["geometry"]["coordinates"][0], f["geometry"]["coordinates"][1])
            date = datetime.strptime(f["properties"]["last_measured_at"], "%Y-%m-%dT%H:%M:%S.%f%z").timestamp()
            lake = False
            if str(f["properties"]["key"]) in lookup:
                lake = lookup[str(f["properties"]["key"])]
            if date > min_date:
                features.append({
                    "type": "Feature",
                    "id": "bafu_" + f["properties"]["key"],
                    "properties": {
                        "label": f["properties"]["label"],
                        "last_time": date,
                        "last_value": float(f["properties"]["last_value"]),
                        "url": "https://www.hydrodaten.admin.ch/en/seen-und-fluesse/stations/{}".format(
                            f["properties"]["key"]),
                        "source": "BAFU Hydrodaten",
                        "icon": "river",
                        "lake": lake
                    },
                    "geometry": {
                        "coordinates": [lng, lat],
                        "type": "Point"}})
except Exception as e:
    print(e)
    failed.append("BAFU")

Thurgau

In [ ]:
try:
    ids = ["M1090", "F1150", "00005"]
    response = requests.get("http://www.hydrodaten.tg.ch/data/internet/layers/30/index.json")
    if response.status_code == 200:
        for f in response.json():
            if f["metadata_station_no"] in ids:
                date = datetime.strptime(f["L1_timestamp"], "%Y-%m-%dT%H:%M:%S.%f%z").timestamp()
                if f["metadata_river_name"] == "":
                    icon = "lake"
                else:
                    icon = "river"
                if date > min_date:
                    features.append({
                        "type": "Feature",
                        "id": "thurgau_" + f["metadata_station_no"],
                        "properties": {
                            "label": f["metadata_station_name"],
                            "last_time": date,
                            "last_value": float(f["L1_ts_value"]),
                            "url": "http://www.hydrodaten.tg.ch/app/index.html#{}".format(f["metadata_station_no"]),
                            "source": "Kanton Thurgau",
                            "icon": icon,
                            "lake": "constance"
                        },
                        "geometry": {
                            "coordinates": [float(f["metadata_station_longitude"]),
                                            float(f["metadata_station_latitude"])],
                            "type": "Point"}})
except Exception as e:
    print(e)
    failed.append("Thurgau")

Zurich

In [ ]:
try:
    stations = {
        "Zürichsee-Oberrieden": {
            "id": "zurich_502",
            "icon": "lake",
            "lake": "zurich",
            "coordinates": [8.584007, 47.272856]},
        "Limmat-Zch. KW Letten": {
            "id": "zurich_578",
            "icon": "river",
            "lake": "zurich",
            "coordinates": [8.531311, 47.387810]},
        "Glatt-Wuhrbrücke": {
            "id": "zurich_531",
            "icon": "river",
            "lake": "greifensee",
            "coordinates": [8.655083, 47.373080]},
        "Türlersee": {
            "id": "zurich_552",
            "icon": "river",
            "lake": "turlersee",
            "coordinates": [8.498728, 47.274612]},
        "Sihl-Blattwag": {
            "id": "zurich_547",
            "icon": "river",
            "lake": False,
            "coordinates": [8.674020, 47.174593]}
    }
    response = requests.get("https://hydroproweb.zh.ch/Listen/AktuelleWerte/AktWassertemp.html")
    if response.status_code == 200:
        df = parse_html_table(response.text.encode('latin-1').decode('utf-8'))
        for index, row in df.iterrows():
            label = row.iloc[0]
            if label in stations:
                date = datetime.strptime(str(row.iloc[3] + row.iloc[2]), "%d.%m.%Y%H:%M")
                date = swiss_timezone.localize(date).timestamp()
                if date > min_date:
                    features.append({
                        "type": "Feature",
                        "id": stations[label]["id"],
                        "properties": {
                            "label": label,
                            "last_time": date,
                            "last_value": float(row.iloc[4]),
                            "url": "https://www.zh.ch/de/umwelt-tiere/wasser-gewaesser/messdaten/wassertemperaturen.html",
                            "source": "Kanton Zurich",
                            "icon": stations[label]["icon"],
                            "lake": stations[label]["lake"]
                        },
                        "geometry": {
                            "coordinates": stations[label]["coordinates"],
                            "type": "Point"}})
except Exception as e:
    print(e)
    failed.append("Zurich")

Datalakes

In [ ]:
try:
    stations = [
        {"id": 1264, "parameters": "y", "label": "Kastanienbaum", "lake": "lucerne"},
        {"id": 515, "parameters": "z?x_index=3", "label": "Greifensee CTD", "lake": "greifensee"},
        {"id": 597, "parameters": "y", "label": "Buchillon", "lake": "geneva"},
        {"id": 448, "parameters": "z?x_index=0", "label": "LéXPLORE Chain", "lake": "geneva"},
        {"id": 1046, "parameters": "z?x_index=0", "label": "Hallwil Chain", "lake": "hallwil"},
        {"id": 956, "parameters": "z?x_index=0", "label": "Murten Chain", "lake": "murten"},
        {"id": 1077, "parameters": "z?x_index=0", "label": "Aegeri Idronaut", "lake": "ageri"},
    ]
    for station in stations:
        response = requests.get(
            "https://api.datalakes-eawag.ch/data/{}/{}".format(station["id"], station["parameters"]))
        if response.status_code == 200:
            data = response.json()
            response = requests.get("https://api.datalakes-eawag.ch/datasets/{}".format(station["id"]))
            if response.status_code == 200:
                metadata = response.json()
                date = datetime.strptime(data["time"], "%Y-%m-%dT%H:%M:%S.%fZ").replace(
                    tzinfo=timezone.utc).timestamp()
                if date > min_date:
                    features.append({
                        "type": "Feature",
                        "id": "datalakes_{}".format(station["id"]),
                        "properties": {
                            "label": station["label"],
                            "last_time": date,
                            "last_value": data["value"],
                            "url": "https://www.datalakes-eawag.ch/datadetail/{}".format(station["id"]),
                            "source": "Datalakes",
                            "icon": "lake",
                            "lake": station["lake"]
                        },
                        "geometry": {
                            "coordinates": [metadata["longitude"], metadata["latitude"]],
                            "type": "Point"}})
except Exception as e:
    print(e)
    failed.append("Datalakes")

Zurich Police

In [ ]:
try:
    stations = [
        {
            "id": "tiefenbrunnen",
            "label": "Tiefenbrunnen",
            "coordinates": [8.561413, 47.348139]
        },
        {
            "id": "mythenquai",
            "label": "Mythenquai",
            "coordinates": [8.536529, 47.357655]
        }
    ]

    for station in stations:
        today = datetime.today()
        startDate = (today - timedelta(days=2)).strftime('%Y-%m-%d')
        endDate = (today + timedelta(days=1)).strftime('%Y-%m-%d')
        response = requests.get("https://tecdottir.herokuapp.com/measurements/{}?startDate={}&endDate={}&sort=timestamp_cet%20desc&limit=1&offset=0".format(station["id"], startDate, endDate))
        if response.status_code == 200:
            data = response.json()
            result = data["result"][0]
            date = datetime.strptime(result["timestamp"], "%Y-%m-%dT%H:%M:%S.%fZ").replace(tzinfo=timezone.utc).timestamp()
            if date > min_date:
                features.append({
                    "type": "Feature",
                    "id": "zurich_police_{}".format(station["id"]),
                    "properties": {
                        "label": station["label"],
                        "last_time": date,
                        "last_value": result["values"]["water_temperature"]["value"],
                        "url": "https://www.tecson-data.ch/zurich/{}/index.php".format(station["id"]),
                        "source": "Stadtpolizei Zürich",
                        "icon": "lake",
                        "lake": "zurich"
                    },
                    "geometry": {
                        "coordinates": station["coordinates"],
                        "type": "Point"}})
except Exception as e:
    print(e)
    failed.append("Zurich Police")

MySwitzerland

In [ ]:
try:
    response = requests.get("https://alplakes-eawag.s3.eu-central-1.amazonaws.com/insitu/myswitzerland.json")
    if response.status_code == 200:
        data = response.json()
        for station in data:
            response = requests.get("https://sospo.myswitzerland.com/lakesides-swimming-pools/{}".format(station))
            if response.status_code == 200:
                root = parse_html(response.text)
                element = html_find_all(root, tag="a", class_name="AreaMap--link")
                location = element[0].get('href').split("?q=")[-1].split(",")
                coords = [float(location[1]), float(location[0])]
                label = html_find_all(root, tag="h1", class_name="PageHeader--title")[0].text
                date = datetime.strptime(html_find_all(root, tag="div", class_name="QuickFactsWidget--info")[0].text.strip().split(": ", 1)[1], "%d.%m.%Y, %H:%M")
                date = swiss_timezone.localize(date).timestamp()
                value = False
                icon = "lake"
                for info in html_find_all(root, tag="ul", class_name="QuickFacts--info"):
                    c = html_find_all(info, tag="li", class_name="QuickFacts--content")
                    if len(c) == 1:
                        content = c[0].text
                        value = html_find_all(info, tag="li", class_name="QuickFacts--value")[0].text
                        if content in ["Lake bathing", "River pools"] and value != "—":
                            if content == "River pools":
                                icon = "river"
                            value = float(value.replace("°", ""))
                if value and date > min_date:
                    features.append({
                    "type": "Feature",
                    "id": "myswitzerland_{}".format(station),
                    "properties": {
                        "label": label,
                        "last_time": date,
                        "last_value": value,
                        "url": "https://sospo.myswitzerland.com/lakesides-swimming-pools/{}".format(station),
                        "source": "MySwitzerland",
                        "icon": icon,
                        "lake": False
                    },
                    "geometry": {
                        "coordinates": coords,
                        "type": "Point"}})
except Exception as e:
    print(e)
    failed.append("MySwitzerland")

In [ ]:
response = requests.get("{}/insitu/summary/water_temperature.geojson".format(bucket))
if response.status_code == 200:
    ids = [f["id"] for f in features]
    old_features = response.json()["features"]
    for f in old_features:
        if f["id"] not in ids and f["properties"]["last_time"] > min_date:
            features.append(f)
            
geojson = {
    "type":"FeatureCollection",
    "name":"Current Water Temperatures",
    "crs":{"type":"name","properties":{"name":"urn:ogc:def:crs:OGC:1.3:CRS84"}},
    "features":features
}

with tempfile.NamedTemporaryFile(mode='w', delete=False) as temp_file:
    temp_filename = temp_file.name
    json.dump(geojson, temp_file)
s3.upload_file(temp_filename, bucket_key, "insitu/summary/water_temperature.geojson")
os.remove(temp_filename)

if len(failed) > 0:
    raise ValueError("Failed for {}".format(", ".join(failed)))